**Download the dataset and annotations**

Download all the images and annotations for cars dataset from [ this link](https://ai.stanford.edu/~jkrause/cars/car_dataset.html).

In [1]:
!wget http://imagenet.stanford.edu/internal/car196/car_ims.tgz
!wget http://imagenet.stanford.edu/internal/car196/cars_annos.mat

--2019-06-18 09:16:58--  http://imagenet.stanford.edu/internal/car196/car_ims.tgz
Resolving imagenet.stanford.edu (imagenet.stanford.edu)... 171.64.68.16
Connecting to imagenet.stanford.edu (imagenet.stanford.edu)|171.64.68.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1956628579 (1.8G) [application/x-gzip]
Saving to: ‘car_ims.tgz.1’

car_ims.tgz.1       100%[===================>]   1.82G  2.66MB/s    in 3m 52s  

2019-06-18 09:20:50 (8.04 MB/s) - ‘car_ims.tgz.1’ saved [1956628579/1956628579]

--2019-06-18 09:20:51--  http://imagenet.stanford.edu/internal/car196/cars_annos.mat
Resolving imagenet.stanford.edu (imagenet.stanford.edu)... 171.64.68.16
Connecting to imagenet.stanford.edu (imagenet.stanford.edu)|171.64.68.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 394471 (385K) [text/plain]
Saving to: ‘cars_annos.mat.1’

cars_annos.mat.1    100%[===================>] 385.23K   785KB/s    in 0.5s    

2019-06-18 09:20:52 (785 KB/s

**Unpack the cars dataset**

In [2]:
import tarfile

print("Extracting the files...")
tf = tarfile.open("car_ims.tgz")
tf.extractall("Cars_Images/")
print("Finished...")

Extracting the files...
Finished...


**Import required libraries to create dataset**

In [0]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import cv2 as cv
import os
from fastai.vision import *
from fastai.callbacks import SaveModelCallback

**Create data**

This will separate all the data into train and test folder.

Also crop the images on bounding boxes given in the annotation file.

In [0]:
mat = scipy.io.loadmat("cars_annos.mat")
annot = mat['annotations'][0]

x1 = []
y1 = []
x2 = []
y2 = []

for i in range(len(annot)):
  
  x1.append(int(annot[i][1]))
  x2.append(int(annot[i][3]))
  y1.append(int(annot[i][2]))
  y2.append(int(annot[i][4]))

try:
  os.mkdir("data")
  os.mkdir("data/train")
  os.mkdir("data/test")
except FileExistsError:
  pass

train_img_name = []
train_lbl = []
test_img_name = []
test_lbl = []

for i in range(len(annot)):
  
  image_name = annot[i][0][0].split("/")[1]
  label = int(annot[i][5])
  
  img = cv.imread("Cars_Images/car_ims/"+image_name)
  
  x_1 = x1[i]
  y_1 = y1[i]
  x_2 = x2[i]
  y_2 = y2[i]
  
  new_img = img[y_1:y_2, x_1:x_2]
  
  flag = int(annot[i][6])
  
  if flag == 0:
    cv.imwrite("data/train/"+image_name, new_img)
    train_img_name.append(image_name)
    train_lbl.append(label)
  else:
    cv.imwrite("data/test/"+image_name, new_img)
    test_img_name.append(image_name)
    test_lbl.append(label)
    
train_csv = pd.DataFrame({'image_names':train_img_name, 'labels':train_lbl})
test_csv = pd.DataFrame({'image_names':test_img_name, 'labels':test_lbl})

train_csv.to_csv("data/train.csv", index=False)

**Helper Function**

get_data(size) --> This will create ImageDataBunch of the given size

train_model(size, i) --> performs couples of operation neccessary to train the classifier

In [0]:
def get_data(size):
  
  """ Creates ImageDataBunch
  """
  
  data = ImageDataBunch.from_csv(path="data/", folder="train",
                              csv_labels="train.csv", test="test",
                              ds_tfms=get_transforms(), size=size, bs=32)
  data.normalize(imagenet_stats)
  return data

In [0]:
def train_model(size, i):
  
  """ Training procedure
  """
  
  learn_50 = cnn_learner(get_data(size), models.resnet50, metrics=[error_rate, accuracy]).load('resnet50_'+str(size))
  learn_50.fit_one_cycle(5*i)
  learn_50.lr_find()
  learn_50.recorder.plot()
  learn_50.unfreeze()
  learn_50.fit_one_cycle(2*i)
  learn_50.save('resnet50_'+str(size))

**Training the model**

first with 64x64 image size, then increasing the image size on top of the previous image size upto 512x512.

In [0]:
image_sizes = [64, 128, 256, 512]

In [0]:
for i, image_size in enumerate(image_sizes):
  
  print("Training model for image_size --> ", image_size)
  
  if i == 0:
    learn_50 = cnn_learner(get_data(image_size), models.resnet50, metrics=[error_rate, accuracy])
    learn_50.save("resnet50_"+str(image_size))
    train_model(image_size, i+1)
  else:
    learn_50 = cnn_learner(get_data(image_size), models.resnet50, metrics=[error_rate, accuracy]).load('resnet50_'+str(image_sizes[i-1]))
    learn_50.save("resnet50_"+str(image_size))
    train_model(image_size, i+1)

In [0]:
for i, image_size in enumerate(image_sizes):
  
  print("Training model for image_size --> ", image_size)
  
  if i == 0:
    pass
  else:
    name = "best_"+str(image_size)
    learn_50 = cnn_learner(get_data(image_size), models.resnet50, metrics=[error_rate, accuracy],
                           callbacks=[SaveModelCallback(learn_50, every='improvement',
                                                        monitor='accuracy', name=name)]).load('resnet50_'+str(image_sizes[i-1]))
    learn_50.fit_one_cycle(10)

In [0]:
learn_50.load('best_512')
log_preds, test_labels = learn_50.get_preds(ds_type=DatasetType.Test)

**Prediction & Confidence for the test dataset**

In [0]:
preds = []
conf = []
classes = np.arange(1, 197)

for log_pred in log_preds:
  
  log_pred = to_np(log_pred)
  val = np.argmax(log_pred)
  preds.append(classes[val])
  conf.append(log_pred[val])

**Function to get prediction on single image**

our model --> learn_50.load("best_512")

In [0]:
def predict_single_img(model, img_path):
  
  """ img_path --> (String) Path to the image
      model --> Fastai model
  """
  
  try:
    img = open_image(img_path)
    x, y, z = model.predict(img)
    classes = np.arange(1, 197)
    z = to_np(z)
    val = np.argmax(z)
    print("Image Class --> ", classes[val])
    print("Confidence --> ", z[val])
  except FileNotFoundError:
    print("File not found")